In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
#para propresar los datos
import category_encoders as ce #(para pasar los datos categoricos a númericos)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
#para reducción de dimensionalidad
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA
#para regresión lineal
from sklearn.linear_model import LinearRegression, Lasso, Ridge
import sklearn.metrics as metrics

Felicidad

In [2]:
df_felicidad = pd.read_csv('felicidad.csv')
df_felicidad.head()

,country,rank,score,high,low,gdp,family,lifexp,freedom,generosity,corruption,dystopia
0,Norway,1,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,Denmark,2,7.522,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
2,Iceland,3,7.504,7.622030,7.385970,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
3,Switzerland,4,7.494,7.561772,7.426227,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
4,Finland,5,7.469,7.527542,7.410458,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182


In [3]:
df_felicidad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   country     155 non-null    object 
 1   rank        155 non-null    int64  
 2   score       155 non-null    float64
 3   high        155 non-null    float64
 4   low         155 non-null    float64
 5   gdp         155 non-null    float64
 6   family      155 non-null    float64
 7   lifexp      155 non-null    float64
 8   freedom     155 non-null    float64
 9   generosity  155 non-null    float64
 10  corruption  155 non-null    float64
 11  dystopia    155 non-null    float64
dtypes: float64(10), int64(1), object(1)
memory usage: 14.7+ KB


In [4]:
df_felicidad_01 = df_felicidad.drop(['country','high','low'],axis=1)
df_felicidad_01.head()

,rank,score,gdp,family,lifexp,freedom,generosity,corruption,dystopia
0,1,7.537,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,2,7.522,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
2,3,7.504,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
3,4,7.494,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
4,5,7.469,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182


sin escalar

In [5]:
#preparamos la data a utilizar
df_felicidad_features = df_felicidad_01.drop(['score'], axis=1)
df_felicidad_target = df_felicidad_01['score']
x_train, x_test, y_train, y_test = train_test_split(df_felicidad_features, df_felicidad_target, test_size=0.2, random_state=42)

escalado

In [6]:
#escalamos los datos
df_felicidad_features_escaled = StandardScaler().fit_transform(df_felicidad_features)
x_train_esc, x_test_esc, y_train_esc, y_test_esc = train_test_split(df_felicidad_features_escaled, df_felicidad_target, test_size=0.2, random_state=42)

Regresión lineal

In [7]:
#sin escalar
reg_lin = LinearRegression()
reg_lin.fit(x_train,y_train)
y_test_pred = reg_lin.predict(x_test)
metrics.mean_squared_error(y_test,y_test_pred)

np.float64(1.0505699820921215e-07)

In [8]:
#escalado
reg_lin.fit(x_train_esc,y_train_esc)
y_test_pred_esc = reg_lin.predict(x_test_esc)
metrics.mean_squared_error(y_test_esc,y_test_pred_esc)

np.float64(1.0505699820929013e-07)

Lasso

In [9]:
#sin escalado
reg_lasso = Lasso(alpha=0.02)
reg_lasso.fit(x_train,y_train)
y_test_pred_lasso = reg_lasso.predict(x_test)
metrics.mean_squared_error(y_test,y_test_pred_lasso)

np.float64(0.020594541925430896)

In [10]:
#escalado
reg_lasso.fit(x_train_esc,y_train_esc)
y_test_pred_lasso_esc = reg_lasso.predict(x_test_esc)
metrics.mean_squared_error(y_test_esc,y_test_pred_lasso_esc)

np.float64(0.015973825208331304)

Ridge

In [11]:
#sin escalado
reg_rdg = Ridge(alpha=1)
reg_rdg.fit(x_train,y_train)
y_test_pred_rdg = reg_rdg.predict(x_test)
metrics.mean_squared_error(y_test,y_test_pred_rdg)

np.float64(0.012316292628182368)

In [12]:
#escalado
reg_rdg.fit(x_train_esc,y_train_esc)
y_test_pred_rdg_esc = reg_rdg.predict(x_test_esc)
metrics.mean_squared_error(y_test_esc,y_test_pred_rdg_esc)

np.float64(0.0005530828083784794)

analisamos las metricas y la mejor aproximación fue la de lineal

Lidiando con outliers

regresiones robustas

In [13]:
df_felicidad_corrup = pd.read_csv('felicidad_corrupt.csv')
df_felicidad_corrup.tail()

,country,rank,score,high,low,gdp,family,lifexp,freedom,generosity,corruption,dystopia
162,H,163,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
163,I,164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
164,J,165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
165,K,166,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
166,L,167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
df_felicidad_02 = df_felicidad_corrup.drop(['country','high','low'],axis=1)
df_felicidad_02.tail()

,rank,score,gdp,family,lifexp,freedom,generosity,corruption,dystopia
162,163,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
163,164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
164,165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
165,166,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
166,167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
from sklearn.linear_model import RANSACRegressor, HuberRegressor
from sklearn.svm import SVR

In [16]:
 #preparamos la data a utilizar
df_felicidad_features_corrup = df_felicidad_02.drop(['score'], axis=1)
df_felicidad_target_corrup = df_felicidad_02['score']
x_train_co, x_test_co, y_train_co, y_test_co = train_test_split(df_felicidad_features_corrup, df_felicidad_target_corrup, test_size=0.2, random_state=42)

In [17]:
#ESTIMADORES 
estimadores = {'SVR':SVR(gamma='auto',C=1.0, epsilon=1.0),'RANSAC': RANSACRegressor(),'HUBER':HuberRegressor(epsilon=1.35)}

In [18]:
for name, estimador in estimadores.items:
    estimador.fit(x_train_co, y_train_co)
    y_test_pred_co = estimador.predict(x_test_co)
    print(name)
    print(metrics.mean_squared_error(y_test_co,y_test_pred_co))

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/paolovf/scikit/env_sci/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_79318/2466951957.py", line 1, in <module>
    for name, estimador in estimadores.items:
TypeError: 'builtin_function_or_method' object is not iterable

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/paolovf/scikit/env_sci/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 2168, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/paolovf/scikit/env_sci/lib/python3.10/site-packages/IPython/core/ultratb.py", line 1454, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/home/paolovf/scikit/env_sci/lib/python3.10/site-packages/IPython/core/ultratb.py", line 1345, in structured_traceback
    return VerboseTB.structur

RANSAC

In [ ]:
reg_ransac = RANSACRegressor()
reg_ransac.fit(x_train_co,y_train_co)
y_test_pred_co_ransac = reg_ransac.predict(x_test_co)
metrics.mean_squared_error(y_test_co,y_test_pred_co_ransac)

np.float64(1.0872387773482723e-07)

HUBER

In [ ]:
reg_hub = HuberRegressor(epsilon=1.35)
reg_hub.fit(x_train_co,y_train_co)
y_test_pred_co_hub = reg_hub.predict(x_test_co)
metrics.mean_squared_error(y_test_co,y_test_pred_co_hub)

/home/paolovf/scikit/env_sci/lib/python3.10/site-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


np.float64(1.93188924110245e-06)

SVM

In [ ]:
svr = SVR(gamma='auto',C=1.0, epsilon=1.0)
svr.fit(x_train_co,y_train_co)
y_test_pred_co_svr = svr.predict(x_test_co)
metrics.mean_squared_error(y_test_co, y_test_pred_co_svr)

np.float64(0.6931459518259915)

concluimos que ransac es el que mejor lidio con los outliers